# Experiments

### Setup

In [1]:
# You can set them inline
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = "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"
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"

In [ ]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

Here is the RAG Application that we've been working with throughout this course

In [4]:
import os
import tempfile
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_openai import OpenAIEmbeddings
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio

# TODO: Configure this model!
MODEL_NAME = "gpt-3.5-turbo"
MODEL_PROVIDER = "openai"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()

def get_vector_db_retriever():
    persist_path = os.path.join(tempfile.gettempdir(), "union.parquet")
    embd = OpenAIEmbeddings()

    # If vector store exists, then load it
    if os.path.exists(persist_path):
        vectorstore = SKLearnVectorStore(
            embedding=embd,
            persist_path=persist_path,
            serializer="parquet"
        )
        return vectorstore.as_retriever(lambda_mult=0)

    # Otherwise, index LangSmith documents and create new vector store
    ls_docs_sitemap_loader = SitemapLoader(web_path="https://docs.smith.langchain.com/sitemap.xml", continue_on_failure=True)
    ls_docs = ls_docs_sitemap_loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500, chunk_overlap=0
    )
    doc_splits = text_splitter.split_documents(ls_docs)

    vectorstore = SKLearnVectorStore.from_documents(
        documents=doc_splits,
        embedding=embd,
        persist_path=persist_path,
        serializer="parquet"
    )
    vectorstore.persist()
    return vectorstore.as_retriever(lambda_mult=0)

nest_asyncio.apply()
retriever = get_vector_db_retriever()

"""
retrieve_documents
- Returns documents fetched from a vectorstore based on the user's question
"""
@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response
- Calls `call_openai` to generate a model response after formatting inputs
"""
@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

"""
call_openai
- Returns the chat completion output from OpenAI
"""
@traceable(
    run_type="llm",
    metadata={
        "ls_provider": MODEL_PROVIDER,
        "ls_model_name": MODEL_NAME
    }
)
def call_openai(messages: List[dict]) -> str:
    return openai_client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
    )

"""
langsmith_rag
- Calls `retrieve_documents` to fetch documents
- Calls `generate_response` to generate a response based on the fetched documents
- Returns the model response
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


### Experiment

Here is a code snippet that should look similar to what you see from the starter code!

There are a few important components here.

1. We have defined an Evaluator
2. We pipe our dataset examples (dict) to the shape of input that our function `langsmith_rag` takes (str) using a target function

In [6]:
from langsmith import evaluate, Client

client = Client()
dataset_name = "RAG Application Golden Dataset"

def is_concise_enough(reference_outputs: dict, outputs: dict) -> dict:
    score = len(outputs["output"]) < 1.5 * len(reference_outputs["output"])
    return {"key": "is_concise", "score": int(score)}

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-3.5-turbo"
)

View the evaluation results for experiment: 'gpt-3.5-turbo-96479868' at:
https://smith.langchain.com/o/7afc0ded-7a59-4bbf-bf6e-24a54c5ee991/datasets/bb9954ef-9c46-4a00-9a44-03680bf5b869/compare?selectedSessions=ca1744a7-d7b9-4a3b-8c46-3edb00e03d41




20it [00:38,  1.90s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,2.043522,130bbf0f-40ed-4f48-bb14-f0211ba5cbc9,20d44a9d-cb01-4471-b66d-419b121434c3
1,How do I pass metadata in with @traceable?,"To pass metadata with `@traceable`, you can in...",None,You can pass metadata with the @traceable deco...,1,3.548707,187ce08a-a5d2-4fbc-ba45-f8456104e334,d41dfb41-a549-44b8-860b-88d87f37de36
2,Can LangSmith be used for finetuning and model...,"LangSmith focuses on monitoring, evaluating, a...",None,"Yes, LangSmith can be used for fine-tuning and...",1,1.485793,3ffc7744-7400-40d0-ba15-e5d355b60af8,e5a214db-b1b1-46da-8f4a-d0e345e43b7f
3,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation thro...",None,"Yes, LangSmith supports online evaluation as a...",0,1.852619,4d107d05-92bd-487c-b546-ac30d7b6a0b5,94c1294a-5f22-4dd2-9d6f-a2c72d4c97c0
4,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,0,2.215705,84c19e52-ff26-4e8d-9b3c-8b32773cc306,2b34b6d8-ba2a-4c11-a0f7-8c1cded6b67c
5,Does LangSmith support offline evaluation?,LangSmith supports online evaluations for real...,None,"Yes, LangSmith supports offline evaluation thr...",1,1.325212,8895fa64-b883-4964-be9a-a6341ea2720d,6f193add-71a1-4088-abc8-377355fc4fb4
6,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, you ne...",None,To trace with the @traceable decorator in Pyth...,1,1.940490,a445fd31-3c87-4247-9850-02c784754900,384b923e-1406-411d-be28-113f2f71a365
7,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.363509,abf65fb9-7f07-407c-998b-938b21d7e126,bb09289e-df56-4b60-a4a6-08cf5067b1ff
8,What is LangSmith used for in three sentences?,LangSmith is used to run multiple experiments ...,None,LangSmith is a platform designed for the devel...,1,1.392018,ca84ac50-3c57-4fca-851c-914f98ea06f6,b5d5dd5a-af31-4e80-be93-15e42ba5c377
9,What testing capabilities does LangSmith have?,LangSmith in LangChain provides testing capabi...,None,LangSmith offers capabilities for creating dat...,1,1.139777,e8d6a7e0-3e90-45f2-9572-ae2f7d495a78,e7d85e73-9588-43bd-b3fe-d51a6754ea07


### Modifying your Application

Now, let's change our model to gpt-35-turbo and see how it performs!

Make this change, and then run this code snippet!

In [7]:
from langsmith import evaluate, Client
from langsmith.schemas import Example, Run

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-3.5-turbo"
)

View the evaluation results for experiment: 'gpt-3.5-turbo-03b9b5dd' at:
https://smith.langchain.com/o/7afc0ded-7a59-4bbf-bf6e-24a54c5ee991/datasets/bb9954ef-9c46-4a00-9a44-03680bf5b869/compare?selectedSessions=0966e2a1-8e28-44e3-ad13-5de6e0498f3d




20it [00:43,  2.17s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,1.841796,130bbf0f-40ed-4f48-bb14-f0211ba5cbc9,48a6e7d6-f983-4af0-9949-d95477b8b87b
1,How do I pass metadata in with @traceable?,"To pass metadata in with @traceable, you can u...",None,You can pass metadata with the @traceable deco...,0,7.880619,187ce08a-a5d2-4fbc-ba45-f8456104e334,ce02693c-f769-4a9b-b977-edca3efc2085
2,Can LangSmith be used for finetuning and model...,LangSmith is primarily designed for observing ...,None,"Yes, LangSmith can be used for fine-tuning and...",0,2.043794,3ffc7744-7400-40d0-ba15-e5d355b60af8,f497da53-42a9-4aa9-a95c-ef348b373c34
3,Does LangSmith support online evaluation?,LangSmith supports online evaluation through i...,None,"Yes, LangSmith supports online evaluation as a...",0,1.480618,4d107d05-92bd-487c-b546-ac30d7b6a0b5,7af97728-1c76-41b7-ae8b-0cc522a10f39
4,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith while using Lan...,None,To set up tracing to LangSmith while using Lan...,1,3.223014,84c19e52-ff26-4e8d-9b3c-8b32773cc306,3453d8e0-8ac9-48d3-8a7b-3aea7ba0a2dc
5,Does LangSmith support offline evaluation?,"LangSmith supports online evaluations, providi...",None,"Yes, LangSmith supports offline evaluation thr...",1,1.943606,8895fa64-b883-4964-be9a-a6341ea2720d,bc94c2bb-ffda-46aa-bcba-6927a7e0f438
6,How can I trace with the @traceable decorator?,To trace with the @traceable decorator in Pyth...,None,To trace with the @traceable decorator in Pyth...,1,1.390151,a445fd31-3c87-4247-9850-02c784754900,d247b394-a6c2-4cbd-9da9-4947a435f477
7,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents ...",None,"Yes, LangSmith can be used to evaluate agents....",0,1.984881,abf65fb9-7f07-407c-998b-938b21d7e126,05c68c7b-8463-44fc-a080-841fd18f6c86
8,What is LangSmith used for in three sentences?,LangSmith is used for running experiments on a...,None,LangSmith is a platform designed for the devel...,1,1.644747,ca84ac50-3c57-4fca-851c-914f98ea06f6,b240fcd0-ca7e-422e-98bf-a45775fa69ac
9,What testing capabilities does LangSmith have?,LangSmith in LangChain provides testing capabi...,None,LangSmith offers capabilities for creating dat...,1,1.571835,e8d6a7e0-3e90-45f2-9572-ae2f7d495a78,4cdb171d-6c94-4089-85e9-a36d02e0e991


### Running over Different pieces of Data

##### Dataset Version

You can execute an experiment on a specific version of a dataset in the sdk by using the `as_of` parameter in `list_examples`

Let's try running on just our initial dataset.

In [8]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name, as_of="initial dataset"),   # We use as_of to specify a version
    evaluators=[is_concise_enough],
    experiment_prefix="initial dataset version"
)

StopIteration: 

##### Dataset Split

You can run an experiment on a specific split of your dataset, let's try running on the Crucial Examples split.

In [9]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name, splits=["Crucial Examples"]),  # We pass in a list of Splits
    evaluators=[is_concise_enough],
    experiment_prefix="Crucial Examples split"
)

StopIteration: 

##### Specific Data Points

You can specify individual data points to run an experiment over as well

In [11]:
evaluate(
    target_function,
    data=client.list_examples(
        dataset_name=dataset_name, 
        example_ids=[   # We pass in a specific list of example_ids
            # TODO: You will need to paste in your own example ids for this to work!
            "130bbf0f-40ed-4f48-bb14-f0211ba5cbc9",
            "187ce08a-a5d2-4fbc-ba45-f8456104e334"
        ]
    ),
    evaluators=[is_concise_enough],
    experiment_prefix="two specific example ids"
)

View the evaluation results for experiment: 'two specific example ids-973bed36' at:
https://smith.langchain.com/o/7afc0ded-7a59-4bbf-bf6e-24a54c5ee991/datasets/bb9954ef-9c46-4a00-9a44-03680bf5b869/compare?selectedSessions=bc9380e3-0b62-4b57-ab62-63f2406710ab




2it [00:04,  2.03s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I create user feedback with the LangSmi...,To create user feedback using the LangSmith SD...,None,To create user feedback with the LangSmith SDK...,1,1.637937,130bbf0f-40ed-4f48-bb14-f0211ba5cbc9,b1d6ed1c-2fe1-448c-ae54-36300ac4fd89
1,How do I pass metadata in with @traceable?,"To pass metadata in with `@traceable`, you can...",None,You can pass metadata with the @traceable deco...,1,1.838410,187ce08a-a5d2-4fbc-ba45-f8456104e334,2ea6715f-bd97-42e5-a091-821bb0610077


### Other Parameters

##### Repetitions

You can run an experiment several times to make sure you have consistent results

In [12]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="two repetitions",
    num_repetitions=2   # This field defaults to 1
)

View the evaluation results for experiment: 'two repetitions-a7f19092' at:
https://smith.langchain.com/o/7afc0ded-7a59-4bbf-bf6e-24a54c5ee991/datasets/bb9954ef-9c46-4a00-9a44-03680bf5b869/compare?selectedSessions=f77abf1d-a515-4cbd-bda9-d7524edf7d4c




40it [01:13,  1.84s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,1.724930,130bbf0f-40ed-4f48-bb14-f0211ba5cbc9,ae6bcadc-9bc3-4429-8f98-e1a238c1e347
1,How do I pass metadata in with @traceable?,"To pass metadata in with @traceable, you can i...",None,You can pass metadata with the @traceable deco...,1,2.264974,187ce08a-a5d2-4fbc-ba45-f8456104e334,feb14a5f-e65a-4362-bd86-83efadb45589
2,Can LangSmith be used for finetuning and model...,LangSmith is primarily focused on observabilit...,None,"Yes, LangSmith can be used for fine-tuning and...",0,2.043235,3ffc7744-7400-40d0-ba15-e5d355b60af8,fbfd7d11-6886-40f8-9ebd-a47333ba4cbe
3,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation thro...",None,"Yes, LangSmith supports online evaluation as a...",0,2.759919,4d107d05-92bd-487c-b546-ac30d7b6a0b5,25e2ce39-284c-46e2-aec9-116e2a8921a4
4,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith using LangChain...,None,To set up tracing to LangSmith while using Lan...,0,2.249280,84c19e52-ff26-4e8d-9b3c-8b32773cc306,60cfaa6a-17a3-42c1-bf48-838ad286c15c
5,Does LangSmith support offline evaluation?,LangSmith primarily supports online evaluation...,None,"Yes, LangSmith supports offline evaluation thr...",1,1.736342,8895fa64-b883-4964-be9a-a6341ea2720d,efadf3cd-8eaa-4e6f-a31b-8ca58ad007c0
6,How can I trace with the @traceable decorator?,To trace with the @traceable decorator in Lang...,None,To trace with the @traceable decorator in Pyth...,1,1.545918,a445fd31-3c87-4247-9850-02c784754900,5d680e01-8822-4cf5-9a95-c01b1728bb5f
7,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.928699,abf65fb9-7f07-407c-998b-938b21d7e126,fe6a9d9e-d835-45c1-b38f-4217fe0e2a4d
8,What is LangSmith used for in three sentences?,LangSmith is used for running experiments on a...,None,LangSmith is a platform designed for the devel...,1,1.818932,ca84ac50-3c57-4fca-851c-914f98ea06f6,bb409b80-08cc-47d8-a3a0-1c8ebb50bfe9
9,What testing capabilities does LangSmith have?,LangSmith in LangChain offers testing capabili...,None,LangSmith offers capabilities for creating dat...,1,1.655926,e8d6a7e0-3e90-45f2-9572-ae2f7d495a78,ba64eb1a-5fec-4d81-b2da-65b73280dab4


##### Concurrency
You can also kick off concurrent threads of execution to make your experiments finish faster!

In [13]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="concurrency",
    max_concurrency=3,  # This defaults to None, so this is an improvement!
)

View the evaluation results for experiment: 'concurrency-8c1a1d9a' at:
https://smith.langchain.com/o/7afc0ded-7a59-4bbf-bf6e-24a54c5ee991/datasets/bb9954ef-9c46-4a00-9a44-03680bf5b869/compare?selectedSessions=f4fab4ce-df59-4ac2-9eac-1dff4c667d63




20it [00:12,  1.62it/s]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Can LangSmith be used for finetuning and model...,LangSmith is primarily designed for monitoring...,None,"Yes, LangSmith can be used for fine-tuning and...",1,1.769748,3ffc7744-7400-40d0-ba15-e5d355b60af8,6de08892-5bcf-4e65-bbfc-e4a633be69c8
1,How do I pass metadata in with @traceable?,"To pass metadata with @traceable, you can incl...",None,You can pass metadata with the @traceable deco...,1,1.806990,187ce08a-a5d2-4fbc-ba45-f8456104e334,2e5dbe0b-50c2-4d9e-9e66-c5ff129c7ad5
2,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,0,2.249529,130bbf0f-40ed-4f48-bb14-f0211ba5cbc9,adfe97a9-fe56-47ee-bda9-8c532e41a828
3,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation thro...",None,"Yes, LangSmith supports online evaluation as a...",0,1.647063,4d107d05-92bd-487c-b546-ac30d7b6a0b5,172b1f5f-2328-40de-b6ab-8d4b3463fb80
4,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,0,1.714566,84c19e52-ff26-4e8d-9b3c-8b32773cc306,4e82aec6-53be-459a-beeb-521f2eec72c9
5,Does LangSmith support offline evaluation?,LangSmith supports online evaluations to provi...,None,"Yes, LangSmith supports offline evaluation thr...",0,1.721501,8895fa64-b883-4964-be9a-a6341ea2720d,6d2f242c-32b9-44a8-9786-85e060e78528
6,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, you ne...",None,To trace with the @traceable decorator in Pyth...,1,1.493197,a445fd31-3c87-4247-9850-02c784754900,20677bac-77e0-4c3d-9f8a-26ae37802c3c
7,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.799148,abf65fb9-7f07-407c-998b-938b21d7e126,e64efa58-faf8-4a3a-bc3c-9a57fa31f765
8,What is LangSmith used for in three sentences?,LangSmith is a tool used for running multiple ...,None,LangSmith is a platform designed for the devel...,1,1.638421,ca84ac50-3c57-4fca-851c-914f98ea06f6,537946b7-4b3a-4967-a101-b2b4497d8708
9,What testing capabilities does LangSmith have?,LangSmith offers the capability to run multipl...,None,LangSmith offers capabilities for creating dat...,1,1.329312,e8d6a7e0-3e90-45f2-9572-ae2f7d495a78,46a5e640-b449-4c1b-8718-ba631159a1f1


##### Metadata 

You can (and should) add metadata to your experiments, to make them easier to find in the UI

In [14]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="metadata added",
    metadata={  # We can pass custom metadata for the experiment, such as the model name
        "model_name": MODEL_NAME
    }
)

View the evaluation results for experiment: 'metadata added-1c5bc4c3' at:
https://smith.langchain.com/o/7afc0ded-7a59-4bbf-bf6e-24a54c5ee991/datasets/bb9954ef-9c46-4a00-9a44-03680bf5b869/compare?selectedSessions=c45c2c25-95f4-4bd5-9c62-f57fc124eb9a




20it [00:33,  1.69s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I create user feedback with the LangSmi...,To create user feedback using the LangSmith SD...,None,To create user feedback with the LangSmith SDK...,1,1.549538,130bbf0f-40ed-4f48-bb14-f0211ba5cbc9,dbcac9da-88dc-472e-bb26-b995775248f7
1,How do I pass metadata in with @traceable?,"To pass metadata in with @traceable, you can i...",None,You can pass metadata with the @traceable deco...,1,1.869545,187ce08a-a5d2-4fbc-ba45-f8456104e334,7b675aae-d3ce-4e95-b45a-220084dc7c8d
2,Can LangSmith be used for finetuning and model...,LangSmith is primarily designed for monitoring...,None,"Yes, LangSmith can be used for fine-tuning and...",1,2.124993,3ffc7744-7400-40d0-ba15-e5d355b60af8,4c1acd09-875b-4c9e-9573-5182f2c11456
3,Does LangSmith support online evaluation?,LangSmith supports online evaluation through i...,None,"Yes, LangSmith supports online evaluation as a...",0,1.460065,4d107d05-92bd-487c-b546-ac30d7b6a0b5,8203c2ed-d822-4a76-8d78-a80491d51d19
4,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,0,1.518580,84c19e52-ff26-4e8d-9b3c-8b32773cc306,a5217586-ca78-41e6-8bff-81eba229fef7
5,Does LangSmith support offline evaluation?,"LangSmith supports online evaluations, providi...",None,"Yes, LangSmith supports offline evaluation thr...",0,1.824090,8895fa64-b883-4964-be9a-a6341ea2720d,32173c28-db19-4d03-8bbd-8d1ead4a066c
6,How can I trace with the @traceable decorator?,You can trace with the @traceable decorator pr...,None,To trace with the @traceable decorator in Pyth...,1,1.869694,a445fd31-3c87-4247-9850-02c784754900,74efc089-5ba9-494c-a70d-44ca4f512ba0
7,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents ...",None,"Yes, LangSmith can be used to evaluate agents....",0,1.577264,abf65fb9-7f07-407c-998b-938b21d7e126,265f4f51-c013-4300-adec-b67c4aaef28b
8,What is LangSmith used for in three sentences?,LangSmith is used to run experiments on datase...,None,LangSmith is a platform designed for the devel...,1,1.396931,ca84ac50-3c57-4fca-851c-914f98ea06f6,139f960a-6ee6-42f4-a396-33eca97b45f4
9,What testing capabilities does LangSmith have?,LangSmith in LangChain offers the capability t...,None,LangSmith offers capabilities for creating dat...,1,1.501247,e8d6a7e0-3e90-45f2-9572-ae2f7d495a78,0b385527-e838-47d5-839c-9d22882cfb91
